# YOLOv8椎体骨折検出モデル学習ノートブック

このノートブックは`train_yolo.py`スクリプトを対話的に実行するためのものです。

## 概要
- YOLOv8を使用した椎体骨折検出モデルの学習
- 医療画像に特化した設定
- Weights & Biasesによる実験管理
- モデルの評価とエクスポート機能

## 1. 環境設定と依存関係のインポート

In [1]:
# 必要なライブラリのインポート
import sys
import os
from pathlib import Path
import torch
import yaml
from datetime import datetime

# プロジェクトルートをパスに追加
project_root = Path(os.getcwd()).parent.parent
sys.path.append(str(project_root))

# train_yolo.pyからクラスをインポート
from train_yolo import VertebralFractureYOLO

print(f"プロジェクトルート: {project_root}")
print(f"CUDA利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA デバイス: {torch.cuda.get_device_name(0)}")
    print(f"CUDA メモリ: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

プロジェクトルート: /mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka
CUDA利用可能: True
CUDA デバイス: NVIDIA RTX A6000
CUDA メモリ: 47.5 GB


## 2. 設定ファイルの確認

In [2]:
# データセット設定ファイルのパス
config_path = project_root / "Sakaguchi_file/YOLO_datasets/vertebrae_fracture/configs/vertebrae_fracture.yaml"

print(f"設定ファイルパス: {config_path}")
print(f"設定ファイル存在: {config_path.exists()}")

# 設定ファイルの内容を表示
if config_path.exists():
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    print("\n設定ファイル内容:")
    for key, value in config.items():
        print(f"  {key}: {value}")
else:
    print("⚠️ 設定ファイルが見つかりません。データセットの準備を確認してください。")

設定ファイルパス: /mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/Sakaguchi_file/YOLO_datasets/vertebrae_fracture/configs/vertebrae_fracture.yaml
設定ファイル存在: True

設定ファイル内容:
  path: /mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/Sakaguchi_file/YOLO_datasets/vertebrae_fracture
  train: train/images
  val: val/images
  test: test/images
  nc: 1
  names: ['fracture']


## 3. YOLOトレーナーの初期化

In [3]:
# 学習パラメータの設定
MODEL_SIZE = 'yolov8m.pt'  # yolov8n.pt, yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt
EPOCHS = 50
BATCH_SIZE = 16
USE_WANDB = True  # Weights & Biasesを使用するかどうか

print(f"モデルサイズ: {MODEL_SIZE}")
print(f"エポック数: {EPOCHS}")
print(f"バッチサイズ: {BATCH_SIZE}")
print(f"Weights & Biases使用: {USE_WANDB}")

# トレーナーを初期化
trainer = VertebralFractureYOLO(
    config_path=str(config_path),
    model_size=MODEL_SIZE
)

print("\n✅ YOLOトレーナーの初期化完了")

2025-07-17 18:28:16,517 - INFO - 設定ファイル読み込み完了: /mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/Sakaguchi_file/YOLO_datasets/vertebrae_fracture/configs/vertebrae_fracture.yaml
2025-07-17 18:28:16,621 - INFO - YOLOv8モデル初期化: yolov8m.pt
2025-07-17 18:28:16,622 - INFO - 出力ディレクトリ: runs/train/20250717_182816


モデルサイズ: yolov8m.pt
エポック数: 50
バッチサイズ: 16
Weights & Biases使用: True

✅ YOLOトレーナーの初期化完了


## 4. Weights & Biasesセットアップ（オプション）

In [ ]:
# wandBでの学習曲線表示設定（S_model_learning準拠）
print("📊 学習曲線はwandBダッシュボードで確認します")
print("💡 以下の手順で学習曲線をリアルタイムで監視できます：")
print()
print("1. 🌐 https://wandb.ai にアクセス")
print("2. 📊 プロジェクト 'vertebrae-fracture-axial-1' を選択")
print("3. 📈 実行中の実験をクリック")
print("4. 📊 'Charts' タブで学習曲線を確認")
print()
print("📈 監視すべき主要メトリクス:")
print("   • train/box_loss - 学習用Box Loss")
print("   • val/box_loss - 検証用Box Loss")
print("   • train/cls_loss - 学習用分類Loss")
print("   • val/cls_loss - 検証用分類Loss")
print("   • metrics/mAP50(B) - mAP@0.5")
print("   • metrics/mAP50-95(B) - mAP@0.5:0.95")
print("   • metrics/precision(B) - 精度")
print("   • metrics/recall(B) - 再現率")
print()
print("💡 学習開始後、wandBダッシュボードで学習曲線がリアルタイムで更新されます")
print("✅ wandB学習曲線監視システム準備完了")

## 5. 学習パラメータの確認

In [5]:
# 学習パラメータを取得して表示
params = trainer.get_training_params()
params['epochs'] = EPOCHS
params['batch'] = BATCH_SIZE

print("学習パラメータ:")
for key, value in params.items():
    print(f"  {key}: {value}")

学習パラメータ:
  data: /mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/Sakaguchi_file/YOLO_datasets/vertebrae_fracture/configs/vertebrae_fracture.yaml
  epochs: 50
  batch: 16
  imgsz: 640
  device: cuda
  lr0: 0.01
  lrf: 0.01
  momentum: 0.937
  weight_decay: 0.0005
  hsv_h: 0.015
  hsv_s: 0.3
  hsv_v: 0.3
  degrees: 20
  translate: 0.1
  scale: 0.2
  shear: 0.1
  perspective: 0.0
  flipud: 0.0
  fliplr: 0.5
  mosaic: 0.8
  mixup: 0.1
  copy_paste: 0.1
  dropout: 0.0
  label_smoothing: 0.0
  val: True
  save_period: 10
  save_json: True
  project: runs/train
  name: 20250717_182816
  patience: 30
  workers: 4
  seed: 42
  deterministic: True
  single_cls: True
  rect: True
  cos_lr: True
  close_mosaic: 10
  resume: False
  amp: True
  fraction: 1.0
  profile: False
  freeze: None
  multi_scale: True
  overlap_mask: True
  mask_ratio: 4
  box: 7.5
  cls: 0.5
  dfl: 1.5
  pose: 12.0
  kobj: 2.0
  nbs: 64
  optimizer: auto


## 6. モデル学習の実行

## 6.3 最終的な学習曲線の表示

In [ ]:
# wandBでの学習結果確認
print("📊 学習結果の確認")
print("=" * 50)

# 学習完了の確認
if 'results' in locals():
    print("✅ 学習が正常に完了しました")
    
    # 学習サマリーの保存
    trainer.save_training_summary(results)
    print("📄 学習サマリーを保存しました")
    
    # wandBでの確認を促す
    print("\\n📊 詳細な学習曲線と結果の確認:")
    print("🌐 https://wandb.ai にアクセス")
    print("📊 プロジェクト: vertebrae-fracture-axial-1")
    print("📈 以下の学習曲線を確認してください:")
    print("   • Loss curves (train/val)")
    print("   • mAP curves")
    print("   • Precision/Recall curves")
    print("   • Learning rate schedule")
    print("   • Model predictions")
    
    # 最終メトリクスの簡易表示
    if hasattr(results, 'results_dict'):
        print("\\n📈 最終メトリクス（概要）:")
        metrics = results.results_dict
        for key, value in metrics.items():
            if 'mAP' in key or 'precision' in key or 'recall' in key:
                print(f"   {key}: {value:.4f}")
    
    # 保存されたモデルの確認
    best_model_path = trainer.output_dir / "weights" / "best.pt"
    last_model_path = trainer.output_dir / "weights" / "last.pt"
    
    print(f"\\n💾 保存されたモデル:")
    if best_model_path.exists():
        print(f"   🏆 Best model: {best_model_path}")
    if last_model_path.exists():
        print(f"   📝 Last model: {last_model_path}")
        
    # YOLOv8が自動生成した結果画像の確認
    results_png = trainer.output_dir / "results.png"
    if results_png.exists():
        print(f"\\n📊 YOLOv8自動生成学習曲線: {results_png}")
        print("   (画像ファイルを直接開いて確認してください)")
    
    print("\\n💡 学習曲線の詳細分析はwandBダッシュボードで行ってください")
    
else:
    print("⚠️ 学習結果が利用できません")
    print("学習を完了してから再実行してください")

In [6]:
# 学習開始時刻を記録
start_time = datetime.now()
print(f"学習開始時刻: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 50)

# 学習実行
try:
    results = trainer.train(epochs=EPOCHS, batch_size=BATCH_SIZE)
    
    # 学習完了
    end_time = datetime.now()
    duration = end_time - start_time
    
    print("=" * 50)
    print(f"✅ 学習完了!")
    print(f"学習時間: {duration}")
    print(f"学習完了時刻: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    
except Exception as e:
    print(f"❌ 学習エラー: {e}")
    raise

2025-07-17 18:13:56,995 - INFO - YOLOv8モデル学習開始


学習開始時刻: 2025-07-17 18:13:56
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.167 🚀 Python-3.12.8 torch-2.4.0+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/Sakaguchi_file/YOLO_datasets/vertebrae_fracture/configs/vertebrae_fracture.yaml, degrees=20, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.3, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=2.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=

100%|██████████| 5.35M/5.35M [00:00<00:00, 86.2MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.2±0.0 ms, read: 86.3±25.7 MB/s, size: 27.8 KB)


train: Scanning /mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/Sakaguchi_file/YOLO_datasets/vertebrae_fracture/train/labels.cache... 37367 images, 33979 backgrounds, 0 corrupt: 100%|██████████| 37367/37367 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
WARNING ⚠️ 'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False
val: Fast image access ✅ (ping: 0.2±0.0 ms, read: 88.8±8.4 MB/s, size: 31.7 KB)


val: Scanning /mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/Sakaguchi_file/YOLO_datasets/vertebrae_fracture/val/labels.cache... 10843 images, 10357 backgrounds, 0 corrupt: 100%|██████████| 10843/10843 [00:00<?, ?it/s]


Plotting labels to runs/train/20250717_1813002/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/train/20250717_1813002
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      13.9G     0.4534      8.413     0.4053          0        544:  38%|███▊      | 892/2336 [02:42<04:23,  5.48it/s]


KeyboardInterrupt: 

## 7. 学習結果の確認

In [ ]:
# 学習結果の表示
if 'results' in locals():
    print("学習結果:")
    if hasattr(results, 'results_dict'):
        for key, value in results.results_dict.items():
            print(f"  {key}: {value}")
    
    # 学習サマリーの保存
    trainer.save_training_summary(results)
    print("\n✅ 学習サマリーを保存しました")
else:
    print("⚠️ 学習結果が利用できません")

## 8. モデルの評価

In [ ]:
# 検証データセットでの評価
try:
    print("検証データセットでの評価を実行中...")
    val_results = trainer.evaluate(data_split='val')
    
    print("\n✅ 評価完了")
    print(f"評価結果: {val_results}")
    
except Exception as e:
    print(f"❌ 評価エラー: {e}")

## 9. サンプル推論（オプション）

In [ ]:
# サンプル画像での推論テスト
# 推論テスト用の画像パスを指定
sample_image_path = None  # 実際のテスト画像パスを指定

if sample_image_path and Path(sample_image_path).exists():
    try:
        print(f"サンプル推論: {sample_image_path}")
        inference_results = trainer.predict_sample(sample_image_path, conf_threshold=0.25)
        print(f"推論結果: {inference_results}")
    except Exception as e:
        print(f"❌ 推論エラー: {e}")
else:
    print("⏭️ サンプル推論をスキップ（画像パスが指定されていません）")

## 10. モデルエクスポート（オプション）

In [ ]:
# モデルをONNX形式でエクスポート
export_format = 'onnx'  # 'onnx', 'torchscript', 'tensorrt'

try:
    print(f"モデルを{export_format}形式でエクスポート中...")
    exported_model = trainer.export_model(format=export_format)
    print(f"✅ エクスポート完了: {exported_model}")
    
except Exception as e:
    print(f"❌ エクスポートエラー: {e}")

## 11. 後処理とクリーンアップ

In [ ]:
# Weights & Biasesセッションの終了
if USE_WANDB:
    try:
        import wandb
        wandb.finish()
        print("✅ Weights & Biases セッション終了")
    except Exception as e:
        print(f"⚠️ Weights & Biases セッション終了エラー: {e}")

# 出力ディレクトリの表示
print(f"\n📁 出力ディレクトリ: {trainer.output_dir}")
print(f"📁 学習結果の保存場所: {trainer.output_dir}")

# 完了メッセージ
print("\n🎉 全ての処理が完了しました！")